# dashboard

In [2]:
import sqlalchemy as db
import pyodbc
import yaml
import seaborn as sns
import pandas as pd


# Standard imports
import pandas as pd

In [3]:


# bokeh
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_8
from bokeh.models import Legend, LegendItem
import itertools

# Panel as well
import panel as pn
pn.extension()
output_notebook()

Loading BokehJS ...

In [4]:
from bokeh.models import CustomJS, Select
from bokeh.io import show
from bokeh.models.formatters import PrintfTickFormatter

In [4]:
# Data to load

In [49]:
# load data
sql = """
SELECT 
ps.[PRIMARY], 
ps.INSDATE,
ps.MA_NR,
ps.WSG_ID,
ps.WZD_ID,
ps.ST_ID,
ps.AT_ID,
ps.WZS_ID ,
ps.KOR_ID ,
ps.PUS_ID ,
IMAX_R,
IMAX_L,
IRMS_R,
IRMS_L,
IINT_R,
IINT_L,
IMAX_R_abHUB2,
IRMS_R_abHUB2,
IINT_R_abHUB2,
POSI_X,
POSI_Y,
POSI_Y_L,
POSI_Z,
ORD1_R,
ORD2_R,
ORDsum_R,
ORD1_L,
ORD2_L,
ORDsum_L,
ORDb_aR,
ORDz1_aR,
ORDz2_aR,
ORDsum_aR,
ORDAmax_aR,
ORDFmax_aR,
ORDb_aL,
ORDz1_aL,
ORDz2_aL,
ORDsum_aL,
ORDAmax_aL,
ORDFmax_aL,
AXIS_C_ACTIVE,
FORCE_FAKTOR,
APM_SCHL_Aktiv,
SCHLEIFZEIT,
IMAX_Xanfahrt,
NIO
FROM dbo.prozessmessung_schleifen ps 
WHERE WARM = 0
AND TEACH_ACTIVE = 0
AND READY = 1
AND MA_NR = 78635
AND WSG_ID IN (14)
AND WZD_ID IN (22)
AND ST_ID IN (20365)
AND AT_ID IN (80);
"""

In [111]:
db_config_file_path = 'C:../db_config.yml'
db_config = yaml.safe_load(open(db_config_file_path))
engine = db.create_engine('{DB_DRIVER}://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?driver={DB_ODBC_DRIVER}'.format(**db_config))

with engine.connect() as conn:
    df = pd.read_sql(sql=sql, con=conn)
    
# add labels
lab = pd.read_csv('../data/labels.csv')
lab.head()
# joining the df with the labels
df = pd.concat([df, lab], axis=1)
df.head()

,PRIMARY,INSDATE,MA_NR,WSG_ID,WZD_ID,ST_ID,AT_ID,WZS_ID,KOR_ID,PUS_ID,...,AXIS_C_ACTIVE,FORCE_FAKTOR,APM_SCHL_Aktiv,SCHLEIFZEIT,IMAX_Xanfahrt,NIO,Unnamed: 0,PRIMARY,labels,scores
0,4303349,2020-09-17 14:18:12,78635,14,22,20365,80,115450,41426,20721,...,2,0.948922,0,28.851,8.523942,0,0,4303349,1,-0.426449
1,4303348,2020-09-17 14:17:42,78635,14,22,20365,80,115450,41426,20721,...,1,0.948922,0,28.879,8.852757,0,1,4303348,1,-0.427286
2,4303347,2020-09-17 14:17:06,78635,14,22,20365,80,115450,41426,20721,...,2,0.948922,0,28.865,8.861331,0,2,4303347,1,-0.420836
3,4303346,2020-09-17 14:16:35,78635,14,22,20365,80,115450,41426,20721,...,1,0.948922,0,28.864,8.947095,0,3,4303346,1,-0.437366
4,4303345,2020-09-17 14:16:00,78635,14,22,20365,80,115450,41426,20721,...,2,0.948922,0,28.861,8.156557,0,4,4303345,1,-0.422145


# Selection panel

# Load Data for machine

In [134]:
# to be replaced with sql query
all_MA_NR = [' 19535', ' 78459', ' 78494', '78635'] # ma_nr_df['MA_NR']


In [192]:
def download_function_machine(attr, old, new): # change name of function
    return print(new)
    #df_model = df.loc[df['MA_NR'] == new] # needs to be changed to function
    #return df_model

sel_machine = Select(title="Select machine number:", value="MANR", options=all_MA_NR, background='grey', margin=0)
sel_machine.on_change("value", download_function_machine)

#show(new_mac)
pn.Column(sel_machine).servable()

Column
    [0] Bokeh(Select)

select combinations of gear geometry etc

In [ ]:
# All possible combinations

# Slider

In [171]:
from bokeh.io import show
from bokeh.models import CustomJS, Slider

slider = Slider(start=0, end=1, value=0.01,  step=.01, width=500, title="Outlier Percentage", format='% ', background='grey', margin=0)
slider.js_on_change("value", CustomJS(code="""
    console.log('slider: value=' + this.value, this.toString())
"""))

show(slider)

# how do I access this value now?

In [ ]:
# Markdown info panel about number of outliers
# defining variables

norm_gear_num = df.loc[df['labels'] == 1].count().sum()
outlier_num = df.loc[df['labels'] == -1].count().sum()

print(norm_gear_num, outlier_num)
print(outlier_num/(norm_gear_num + outlier_num)*100)

In [143]:
# count_info = pn.pane.Markdown('''
# 
## 
# <br>

# ''', background='grey', margin=0)

In [ ]:
def callback(s, m):
    

# Servable

In [169]:
header = pn.pane.Markdown('''
# Reishauer
## Anomaly Detection
<br>

''', background='grey', margin=0)


In [184]:
plot_variable = pn.widgets.Select(name='Select Plot Variable', options=['PRIMARY', 'ID', 'INSDATE', 'MA_NR', 'WSG_ID', 'WZS_ID', 'WZD_ID',
       'KOR_ID', 'ST_ID', 'AT_ID', 'PUS_ID', 'IMAX_R', 'IMAX_L', 'IRMS_R',
       'IRMS_L', 'IINT_R', 'IINT_L', 'IMAX_R_abHUB2', 'IRMS_R_abHUB2',
       'IINT_R_abHUB2', 'POSI_X', 'POSI_Y', 'POSI_Y_L', 'POSI_Z', 'ORD1_R',
       'ORD2_R', 'ORDz1_I_R', 'ORDz2_I_R', 'ORDz3_I_R', 'ORDsum_R', 'ORD1_L',
       'ORD2_L', 'ORDz1_I_L', 'ORDz2_I_L', 'ORDz3_I_L', 'ORDsum_L', 'ORDb_aR',
       'ORDz1_aR', 'ORDz2_aR', 'ORDz3_aR', 'ORDsum_aR', 'ORDAmax_aR',
       'ORDFmax_aR', 'ORDb_aL', 'ORDz1_aL', 'ORDz2_aL', 'ORDz3_aL',
       'ORDsum_aL', 'ORDAmax_aL', 'ORDFmax_aL', 'AXIS_C_ACTIVE',
       'FORCE_FAKTOR', 'APM_SCHL_Aktiv', 'SCHLEIFZEIT', 'IRMS_V',
       'IMAX_Xanfahrt', 'IINT_V', 'VORSTUFE_AKTIV', 'IMAX_V', 'IINTOFF_R',
       'NIO', 'NIO_GROUP'])

y=plot_variable.value
y_outlier=df[['INSDATE',y]].loc[df['labels']==-1]

def get_plot():
    from bokeh.plotting import figure, show

    # output_file("line.html")
    cds = ColumnDataSource(df_model)
    p = figure(plot_width=1000, plot_height=400, x_axis_type="datetime")

    # add a line renderer
    p.line('INSDATE', y, source=cds, line_width=2)
    p.circle(y_outlier['INSDATE'], y_outlier[y], color="red")
    return p

# Create grid